## Investigating different cost functions

In this notebook, we take a look at the different cost function offered in PyBOP. Cost functions conventionally construct a distance metric between two mathematics sets (vectors), which is then used within PyBOP's optimisation algorthims. 

First, we install and import the required packages below.

In [80]:
%pip install --upgrade pip ipywidgets
%pip install pybop -q

import numpy as np
import plotly.graph_objects as go

import pybop

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


For this notebook, we need to construct parameters, a model and a problem class before we can compare differing cost functions. We start with two parameters, but this is an arbituary selection and can be expanded given the model and data in question.

In [81]:
parameters = pybop.Parameters(
    pybop.Parameter(
        "Positive electrode thickness [m]",
        prior=pybop.Gaussian(7.56e-05, 0.5e-05),
        bounds=[65e-06, 10e-05],
    ),
    pybop.Parameter(
        "Positive particle radius [m]",
        prior=pybop.Gaussian(5.22e-06, 0.5e-06),
        bounds=[2e-06, 9e-06],
    ),
)

Next, we will construct the Single Particle Model (SPM) with the Chen2020 parameter set, but like the above, this is an arbitruary selection and can be replaced with any PyBOP model.

In [82]:
parameter_set = pybop.ParameterSet.pybamm("Chen2020")
model = pybop.lithium_ion.SPM(parameter_set=parameter_set)

Next, as we will need reference data to compare our model predictions to (via the cost function), we will create synthetic data from the model constructed above. 

In [83]:
t_eval = np.arange(0, 900, 2)
values = model.predict(t_eval=t_eval)

We can then construct the PyBOP dataset class with the synthetic data as,

In [84]:
dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": values["Current [A]"].data,
        "Voltage [V]": values["Voltage [V]"].data,
    }
)

Now, we can put this all together and construct the problem class. In this situation, we are going to compare differing fitting cost functions, so we construct the `FittingProblem`.

In [85]:
problem = pybop.FittingProblem(model, parameters, dataset)

### Sum of Square Errors and Root Mean Squared Error

First, let's start with the easiest cost functions, the sum of squared errors (SSE) and the root mean squared error (RMSE). Constructing these classes is very concise in PyBOP, and only requires the problem class.

In [86]:
cost_SSE = pybop.SumSquaredError(problem)
cost_RMSE = pybop.RootMeanSquaredError(problem)

Now, we can investigate how these functions differ when fitting the parameters. To acquire the distance metric for each of these, we can simply use the constructed class in a call method, such as:

In [87]:
cost_SSE([7.56e-05, 5.22e-06])

1.2690291451182834e-09

Alternatively, we can use the `Parameters` class for this,

In [88]:
print(parameters.current_value())
cost_SSE(parameters.current_value())

[7.56e-05, 5.22e-06]


1.2690291451182834e-09

If we want to generate a random sample of candidate solutions from the parameter class prior, we can also do that as:

In [89]:
sample = parameters.rvs()
print(sample)
cost_SSE(sample)

[7.60957550e-05 5.48691392e-06]


0.014466013735507651

#### Comparing RMSE and SSE

Now, let's vary one of the parameters with a fixed point for the other and create a scatter plot comparing the cost values for the RMSE and SSE functions.

In [90]:
x_range = np.linspace(4.8e-06, 5.6e-06, 75)
y_SSE = []
y_RMSE = []
for i in x_range:
    y_SSE.append(cost_SSE([7.56e-05, i]))
    y_RMSE.append(cost_RMSE([7.56e-05, i]))

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_range, y=y_SSE, mode="lines", name="SSE"))
fig.add_trace(go.Scatter(x=x_range, y=y_RMSE, mode="lines", name="RMSE"))
fig.show()

In this situation, it's clear that gradient of the SSE cost is much higher than the RMSE. This can be helpful for certain optimisation algorithms, specifically towards improving convergence performance within a predefine number of iterations. However, with incorrect hyperparameter values this can also result in the algorithm not converging due to sampling locations outside of the "cost valley".

### Minkowski Cost

Next, we will investigate using the Minkowski cost function. This cost function provides a general formation of the above two cost functions, allowing for hyper parameter calibration on the cost function itself. The Minkowski cost takes the form,

$\mathcal{L} = \displaystyle\sum_{1}^N  (\hat{y}-y)^p$

For p = 1, this becomes L1Norm  
For p = 2, this becomes L2Norm (SSE)

PyBOP offers a Minkowski class, which we will construct below. This class has an optional argument of `p` which designates the order in the above equation. This value can be a float, with the only requirement that it is not negative. First, let's reconstruct the SSE function with a `p` value of 2.

In [91]:
cost_minkowski = pybop.Minkowski(problem, p=2)

In [92]:
y_minkowski = []
for i in x_range:
    y_minkowski.append(cost_minkowski([7.56e-05, i]))

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_range, y=y_SSE, mode="lines", name="SSE"))
fig.add_trace(go.Scatter(x=x_range, y=y_minkowski, mode="lines", name="Minkowski"))
fig.show()

As expected, these cost functions are equivalent. Now, let take a look at how the Minkowski cost changes for different orders of `p`.

In [93]:
y_minkowski = ()
p_orders = np.append(np.linspace(1, 3, 5), 0.75)
for j in p_orders:
    minkowski_order = []
    cost = pybop.Minkowski(problem, p=j)
    for i in x_range:
        minkowski_order.append(cost([7.56e-05, i]))
    y_minkowski += (minkowski_order,)

In [94]:
fig = go.Figure()
for k, _ in enumerate(p_orders):
    fig.add_trace(
        go.Scatter(x=x_range, y=y_minkowski[k], mode="lines", name=f"Minkowski {_}")
    )
fig.show()

As seen above, the Minkowski cost function allows for a variety of different distance metrics to be created. This provides users with another hyper parameter for to calibrate for optimisation algorithm convergence. This addition does expand the global search space, and should be carefully considered before deciding upon.

In this notebook, we've shown the different distance metrics (cost functions) offered in PyBOP. Selection between these functions can effect the identified parameters in the case that the optimiser hyperparameter values are not properly calibrated. 